# LangGraph Advanced – Part 4️⃣: Inline LoRA Fine‑Tuning Node

Sử dụng PEFT LoRA trong node LangGraph.

In [ ]:
%pip install -q langgraph transformers peft datasets accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from langgraph.graph import StateGraph, MessageState
import torch

BASE_MODEL = "facebook/opt-350m"
token = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_8bit=True, device_map="auto")
model = prepare_model_for_kbit_training(model)
lora_cfg = LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj","v_proj"])
model = get_peft_model(model, lora_cfg)

class LoRAState(MessageState): pass

def train(state: LoRAState):
    ds = load_dataset("yelp_review_full", split="train[:0.1%]")
    def tokenize(batch):
        return token(batch["text"], truncation=True, padding="max_length", max_length=64)
    ds = ds.map(tokenize, batched=True)
    args = TrainingArguments(output_dir="lora_out", per_device_train_batch_size=4,
                             num_train_epochs=1, fp16=True)
    model.train()
    trainer = torch.compile
    return state

g = StateGraph(LoRAState).add_node("train", train).set_entry_point("train").compile()
